# SUBMISSION GENERATOR

## 1-IMPORTS

In [ ]:
import json
import os
import numpy as np
from submission_generator_tools import *
from AxonDeepSeg.config_tools import *
import time

## 2-DEFINING USEFUL FUNCTIONS

In [ ]:
########## HEADER ##########
# Config file description :

# network_learning_rate : float : No idea, but certainly linked to the back propagation ? Default : 0.0005.
# network_n_classes : int : number of labels in the output. Default : 2.
# network_dropout : float : between 0 and 1 : percentage of neurons we want to keep. Default : 0.75.
# network_depth : int : number of layers WARNING : factualy, there will be 2*network_depth layers. Default : 6.
# network_convolution_per_layer : list of int, length = network_depth : number of convolution per layer. Default : [1 for i in range(network_depth)].
# network_size_of_convolutions_per_layer : list of lists of int [number of layers[number_of_convolutions_per_layer]] : Describe the size of each convolution filter.
# Default : [[3 for k in range(network_convolution_per_layer[i])] for i in range(network_depth)].
# network_features_per_layer : list of lists of int [number of layers[number_of_convolutions_per_layer[2]] : Numer of different filters that are going to be used.
# Default : [[64 for k in range(network_convolution_per_layer[i])] for i in range(network_depth)]. WARNING ! network_features_per_layer[k][1] = network_features_per_layer[k+1][0].
# network_trainingset : string : describe the trainingset for the network.
# network_downsampling : string 'maxpooling' or 'convolution' : the downsampling method.
# network_thresholds : list of float in [0,1] : the thresholds for the ground truthes labels.
# network_weighted_cost : boolean : whether we use weighted cost for training or not.


## 3-GENERATING THE FILES

Defining the name of the files and models, and environment variables.

**IMPORTANT TO CHANGE AT EACH GENERATION**

In [ ]:
server = 'Helios' # Choose between Guillimin and Helios

# Warning: model names must finish by "/"

path_models = '../models/'
path_data = '../../../trainingsets/'

** PARAMETERS SELECTION **

Select what parameters you want to change compared to the default ones. If you want to try several values for a parameter, just use a list for it in dict_params.

In [ ]:
L_struct = [{'structure':[[5,5], [3,3], [3,3], [3,3]], 'features_augmentation':'x2', 'network_first_num_features':16}]

dict_params = {'network_batch_norm_decay':0.7,
              'network_additional_parameters':[{'batch_norm_decay_ending_decay':0.999,
                                               'batch_norm_decay_decay_period':2000,
                                               'batch_norm_decay_decay_activate':False,
                                               'learning_rate_decay_activate':True,
                                               'learning_rate_decay_period':120
                                              },
                                              {'batch_norm_decay_ending_decay':0.999,
                                               'batch_norm_decay_decay_period':2000,
                                               'batch_norm_decay_decay_activate':False,
                                               'learning_rate_decay_activate':True,
                                               'learning_rate_decay_period':200
                                              },
                                              {'batch_norm_decay_ending_decay':0.999,
                                               'batch_norm_decay_decay_period':2000,
                                               'batch_norm_decay_decay_activate':False,
                                               'learning_rate_decay_activate':True,
                                               'learning_rate_decay_period':240
                                              },
                                              {'batch_norm_decay_ending_decay':0.999,
                                               'batch_norm_decay_decay_period':2000,
                                               'batch_norm_decay_decay_activate':False,
                                               'learning_rate_decay_activate':True,
                                               'learning_rate_decay_period':80
                                              },
                                              {'batch_norm_decay_ending_decay':0.999,
                                               'batch_norm_decay_decay_period':2000,
                                               'batch_norm_decay_decay_activate':False,
                                               'learning_rate_decay_activate':True,
                                               'learning_rate_decay_period':400
                                              }
                                              ]
              }


config_list = grid_config(L_struct, dict_params)

**Options**

In [ ]:
walltime_hours = 12 # Change here the duration of each task

** SUBMISSION FILES GENERATION **

In [ ]:
if server == 'Helios':
    path_project = 'maxime'
    path_venv = '/home/maxwab/tf11-py27'
elif server == 'Guillimin':
    path_project = '/gs/project/rrp-355-aa/maxwab'
    path_venv = '~/maxwab/tf11-py27'
else:
    raise ValueError("Error, no correct server selected")
    
# Maxime values :

#if server == 'Helios':
#    path_project = 'maxime'
#    path_venv = '/home/maxwab/tf11-py27'
#elif server == 'Guillimin':
#    path_project = '/gs/project/rrp-355-aa/maxwab'
#    path_venv = '~/maxwab/tf11-py27'
    

We now generate the bash file that will launch all submissions at once, as well as the required sh files.

In [ ]:
file = open(path_models + "/global_submission.sh","w") # We create the general submission file at the root of the models folder

for config_name,config in config_list.iteritems():

    submission_filename = "submission.sh"
    if server == 'Helios':
        file.write('msub ')
    elif server == 'Guillimin':
        file.write('qsub ')
    file.write(str(config_name)+'/')
    file.write(str(submission_filename))
    if server == 'Guillimin':
        file.write(' -l nodes=1:gpus=1:exclusive_process') # To avoid a server error. Check that the number of nodes is the good one !
    file.write('\n')
    
    if server == 'Helios':
        generate_heliosjob(path_project, path_venv, submission_filename, 'config_network.json', config,
                       path_trainingset = path_data+config['network_trainingset']+'/training/',
                       path_model = path_models+config_name, walltime = 3600*walltime_hours-30
                      )
    elif server == 'Guillimin':
        generate_guilliminjob(path_project, path_venv, submission_filename, 'config_network.json', config,
                       path_trainingset = path_data+config['network_trainingset']+'/training/',
                       path_model = path_models+config_name, walltime = 3600*walltime_hours-30
                      )

file.close()